In [3]:
!pip install tensorflow-text

**Importing importants libraries**

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.patches as patches
from collections import Counter
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Reading Train Data**

In [6]:
data=pd.read_csv("/content/drive/MyDrive/Doceree-Complete-DataSet/Doceree_Complete_DataSet/Doceree-HCP_Train.csv",encoding='latin1')

In [7]:
data.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal,Website,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0


In [8]:
data.nunique()

ID                 113937
DEVICETYPE              4
PLATFORM_ID            15
BIDREQUESTIP        33664
USERPLATFORMUID     53780
USERCITY             4420
USERZIPCODE         11278
USERAGENT            4288
PLATFORMTYPE            5
CHANNELTYPE             1
URL                  5231
KEYWORDS             2460
TAXONOMY              207
IS_HCP                  2
dtype: int64

In [9]:
data.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            6359
USERZIPCODE         4592
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81624
IS_HCP                 1
dtype: int64

In [10]:
data= data[data.IS_HCP!=0]

In [11]:
data.shape

(34181, 14)

In [12]:
data

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
8,1009,Mobile,2,104.172.116.202,be61a8f7-0f00-487f-8a56-679c819fa01a,Granada Hills,91344.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like M...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Intensive Care|Shock|Ischemic Heart Disease|He...,2084P0804X,1.0
21,1022,Desktop,3,108.217.77.216,6d0996e0-b6f4-4bb8-ae40-13ee2ae10706,NaN,912031130.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/protein-require...,Gastrointestinal Tract|Female|Oncology|Medicin...,2084N0400X,1.0
29,1030,Desktop,7,23.125.217.54,18783e3e-89b3-4b50-a143-231bc9e235f2,Buena Park,90620.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,Online Medical Journal,Website,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,2084V0102X,1.0
35,1036,Desktop,2,73.4.155.15,cca6db3d-9041-4666-904a-6626154459da,Somerville,2143.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.rheumatologyadvisor.com/home/decis...,Dermatomyositis|Preventive|Male|Dermatology|Ag...,207RC0000X,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113932,114933,Desktop,2,68.82.97.126,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,Philadelphia,19120.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/consultat...,Cardiovascular|Clinical|False|Male|Oncology|Ps...,207RP1001X,1.0
113933,114934,Desktop,2,104.172.11.109,3289bba5-4a50-4b61-a5cc-68596bbdc2bf,Van Nuys,914014213.0,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/features/...,Asthma|Arthritis|General|Clinical|Diagnosis|Rh...,2084P0800X,1.0
113934,114935,Desktop,7,174.21.94.113,9c36e90c-caa7-4323-8431-fb84f8293972,NaN,98105.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,208000000X,1.0
113935,114936,Mobile,2,69.253.129.131,afc27760-31f7-4f6e-aa08-90d8a96525d0,Wilmington,19808.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.dermatologyadvisor.com/home/topics...,pain|disease|autoimmune|multiple sclerosis|gra...,2084P0800X,1.0


In [13]:
data=data[['KEYWORDS','TAXONOMY']]

In [14]:
data.isnull().sum()

KEYWORDS       0
TAXONOMY    1868
dtype: int64

In [15]:
data.describe

<bound method NDFrame.describe of                                                  KEYWORDS    TAXONOMY
3       Dental|Total|Clinical|Pharmacology|Physicians|...  2084P0800X
8       Intensive Care|Shock|Ischemic Heart Disease|He...  2084P0804X
21      Gastrointestinal Tract|Female|Oncology|Medicin...  2084N0400X
29      Rehabilitation|Transplantation|Clinical|Preven...  2084V0102X
35      Dermatomyositis|Preventive|Male|Dermatology|Ag...  207RC0000X
...                                                   ...         ...
113932  Cardiovascular|Clinical|False|Male|Oncology|Ps...  207RP1001X
113933  Asthma|Arthritis|General|Clinical|Diagnosis|Rh...  2084P0800X
113934  Rehabilitation|Transplantation|Clinical|Preven...  208000000X
113935  pain|disease|autoimmune|multiple sclerosis|gra...  2084P0800X
113936  Genetic|Large|Autoimmune|Health|False|Abortion...  2084P0800X

[34181 rows x 2 columns]>

**Selecting important features for prediction**

In [69]:
# Create a new DataFrame to store the modified data
new_df = pd.concat([data] + [data['KEYWORDS'].str.get_dummies(sep='|').add_prefix('Keyword_')], axis=1)

# Drop the original KEYWORDS column
new_df.drop('KEYWORDS', axis=1, inplace=True)

In [70]:
new_df.shape

(34181, 1261)

**fillna value with Most frequent values**

In [71]:
new_df['TAXONOMY']=new_df['TAXONOMY'].fillna(new_df['TAXONOMY'].mode()[0])

In [72]:
new_df.isnull().sum()

TAXONOMY                       0
Keyword_Ablation Techniques    0
Keyword_Abortion               0
Keyword_Abscess                0
Keyword_Acid-Base Imbalance    0
                              ..
Keyword_wound infection        0
Keyword_yaws                   0
Keyword_yellow fever           0
Keyword_zoonoses               0
Keyword_zoster sine herpete    0
Length: 1261, dtype: int64

In [73]:
target=new_df['TAXONOMY']

In [74]:
new_df.drop('TAXONOMY',axis=1,inplace=True)

In [75]:
new_df.shape

(34181, 1260)

In [27]:
type(new_df)

pandas.core.frame.DataFrame

**Training Model with train data**

In [76]:
import tensorflow as tf

In [77]:
x_train, x_test, y_train, y_test = train_test_split(new_df, target, test_size=0.1, random_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(30762, 1260)
(3419, 1260)
(30762,)
(3419,)


In [78]:
model = RandomForestClassifier(n_estimators=100, random_state=0)
with tf.device('/gpu:0'):
  # Fit the model to the training data
  model.fit(x_train, y_train)

In [79]:
y_predict = model.predict(x_test)
type(y_predict)

numpy.ndarray

In [80]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy:", accuracy)

Accuracy: 0.40333430827727407


In [81]:
y_predict

array(['390200000X', '207RG0100X', '207Q00000X', ..., '2084P0800X',
       '2084P0800X', '2084P0800X'], dtype=object)

**Reading Test data file**

In [86]:
data_test=pd.read_csv("/content/drive/MyDrive/Doceree-Complete-DataSet/Doceree_Complete_DataSet/Doceree-HCP_Test.csv",encoding='latin1')

In [83]:
data_test


,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS
0,115501,Desktop,2,75.189.231.103,0d5041ff-f0b6-4d1a-9ad7-0a29f7d485b4,Fayetteville,28305.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...
1,115502,Mobile,2,24.101.33.158,c8396dd0-969f-4d99-a40b-b7bb1f516154,Conneaut Lake,16316.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.ophthalmologyadvisor.com/topics/ca...,General|Clinical|Operative|Medicine|Cardiology...
2,115503,Desktop,2,172.118.216.142,3c97a081-6518-43f8-9f26-369759cfb471,Covina,91724.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.psychiatryadvisor.com/author/tori-...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...
3,115504,Desktop,7,71.105.120.171,3e2578c8-f794-41af-a38c-c5cfb3c0f014,Brooklyn,11226.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/105482-diverti...,Health|Male|Neurological Surgery|Otolaryngolog...
4,115505,Desktop,2,73.82.211.73,ec2ae7ce-6a8c-4156-98a7-07203e60f483,Marietta,30062.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.renalandurologynews.com/home/confe...,chronic kidney disease|pain|nephrology|disease...
...,...,...,...,...,...,...,...,...,...,...,...,...
28488,143989,Desktop,2,69.202.233.241,78ce4bbe-3885-4c14-b945-a0ea1e4574f4,Brooklyn,11215.0,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,Online Medical Journal,Website,https://www.renalandurologynews.com/home/news/...,Transplantation|Psychiatry|Heart Failure|Angio...
28489,143990,Desktop,7,75.4.190.65,a53799e1-b279-40cf-b467-11b4d659e325,Miami,33178.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cureus.com/publish/articles/144818...,small|hospitals|nephrology|emergency medicine|...
28490,143991,Desktop,7,137.52.180.45,acc732a8-2ee9-4e2c-9933-70e34db48101,Fort Lauderdale,33314.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/registrations/continue?...,small|hospitals|nephrology|emergency medicine|...
28491,143992,Desktop,8,66.249.66.4,dce21294-b105-4abb-b145-4e62d71def44,NaN,NaN,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,Online Medical Journal,Website,https://radrounds.com/radiology-case-images-te...,the|disease|pain|radiology|lung abscess|dyspar...


In [84]:
data_test.isnull().sum()

ID                    0
DEVICETYPE            0
PLATFORM_ID           0
BIDREQUESTIP          0
USERPLATFORMUID       0
USERCITY           1559
USERZIPCODE        1115
USERAGENT             1
PLATFORMTYPE          0
CHANNELTYPE           0
URL                   0
KEYWORDS              0
dtype: int64

In [89]:
data_test=data_test[['KEYWORDS']]

In [90]:
data_test

,KEYWORDS
0,Family Practice|Drainage|Clinical|Dermatology|...
1,General|Clinical|Operative|Medicine|Cardiology...
2,Abortion|Anxiety Disorders|Apnea|False|Trauma|...
3,Health|Male|Neurological Surgery|Otolaryngolog...
4,chronic kidney disease|pain|nephrology|disease...
...,...
28488,Transplantation|Psychiatry|Heart Failure|Angio...
28489,small|hospitals|nephrology|emergency medicine|...
28490,small|hospitals|nephrology|emergency medicine|...
28491,the|disease|pain|radiology|lung abscess|dyspar...


In [91]:
new_test_data = pd.concat([data_test] + [data_test['KEYWORDS'].str.get_dummies(sep='|').add_prefix('Keyword_')], axis=1)
new_test_data.drop('KEYWORDS', axis=1, inplace=True)

In [93]:
new_test_data.shape

(28493, 1426)

**Finding common features**

In [94]:
a=new_df.columns
print(a.shape)

b=new_test_data.columns
print(b.shape)

(1260,)
(1426,)


In [95]:
c=a.intersection(b)
type(c)

pandas.core.indexes.base.Index

In [96]:
c.shape


(1140,)

In [97]:
new_df=new_df[c]

**Training model with common features**

In [98]:
x_train, x_test, y_train, y_test = train_test_split(new_df, target, test_size=0.1, random_state=0)

In [99]:
model = RandomForestClassifier(n_estimators=100, random_state=0)
with tf.device('/gpu:0'):
  # Fit the model to the training data
  model.fit(x_train, y_train)

In [100]:
y_predict = model.predict(x_test)


In [101]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy:", accuracy)

Accuracy: 0.40304182509505704


In [102]:
new_test_data=new_test_data[c]
new_test_data.shape

(28493, 1140)

In [103]:
y_predict_test=model.predict(new_test_data)
y_predict_test

array(['2084P0800X', '2084P0800X', '2084P0800X', ..., '2084P0800X',
       '367A00000X', '2084P0800X'], dtype=object)

**Using Scoring solution predicted file which is predicted in Scoring source code**

In [104]:
scoring_solution=pd.read_csv("/content/ans (2).csv")

In [105]:
scoring_solution

,ID,IS_HCP
0,115501,0.0
1,115502,1.0
2,115503,0.0
3,115504,0.0
4,115505,1.0
...,...,...
28488,143989,0.0
28489,143990,0.0
28490,143991,0.0
28491,143992,0.0


In [107]:
type(y_predict_test)
TAXONOMY=pd.Series(y_predict_test)
type(TAXONOMY)

pandas.core.series.Series

In [109]:
scoring_solution['TAXONOMY']=TAXONOMY.values

In [110]:
scoring_solution

,ID,IS_HCP,TAXONOMY
0,115501,0.0,2084P0800X
1,115502,1.0,2084P0800X
2,115503,0.0,2084P0800X
3,115504,0.0,2084N0400X
4,115505,1.0,2084N0400X
...,...,...,...
28488,143989,0.0,2084P0800X
28489,143990,0.0,390200000X
28490,143991,0.0,2084P0800X
28491,143992,0.0,367A00000X


**Saving file into complete solution**

In [111]:
scoring_solution.to_csv('complete_solution.csv',index=False)